In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

24/06/04 21:50:58 WARN Utils: Your hostname, SMB.local resolves to a loopback address: 127.0.0.1; using 10.0.0.125 instead (on interface en0)
24/06/04 21:50:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/04 21:50:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')

In [4]:
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("sales")


In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

spark.sql("""SELECT YEAR(date) AS year,
          ROUND(avg(price), 2) AS annual_avg_price
          FROM sales
          WHERE bedrooms = 4
          GROUP BY year
          ORDER BY year ASC""").show()

+----+----------------+
|year|annual_avg_price|
+----+----------------+
|2019|        300263.7|
|2020|       298353.78|
|2021|       301819.44|
|2022|       296363.88|
+----+----------------+



In [7]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""SELECT date_built,
          ROUND(avg(price), 2) AS annual_avg_price
          FROM sales
          WHERE bedrooms = 3
          AND bathrooms = 3
          GROUP BY date_built
          ORDER BY date_built ASC""").show()


+----------+----------------+
|date_built|annual_avg_price|
+----------+----------------+
|      2010|       292859.62|
|      2011|       291117.47|
|      2012|       293683.19|
|      2013|       295962.27|
|      2014|       290852.27|
|      2015|        288770.3|
|      2016|       290555.07|
|      2017|       292676.79|
+----------+----------------+



In [8]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""SELECT date_built,
          ROUND(avg(price), 2) AS annual_avg_price
          FROM sales
          WHERE bedrooms = 3
          AND bathrooms = 3 
          AND floors = 2
          AND sqft_living >= 2000
          GROUP BY date_built
          ORDER BY date_built ASC""").show()


+----------+----------------+
|date_built|annual_avg_price|
+----------+----------------+
|      2010|       285010.22|
|      2011|       276553.81|
|      2012|       307539.97|
|      2013|       303676.79|
|      2014|       298264.72|
|      2015|       297609.97|
|      2016|        293965.1|
|      2017|       280317.58|
+----------+----------------+



In [9]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT
view, 
ROUND(AVG(price / view), 2) AS avg_price_per_view
FROM sales
GROUP BY view
HAVING AVG(price) > 350000
ORDER BY avg_price_per_view DESC""" ).show()

print(f"--- {(time.time() - start_time)} seconds ---")

+----+------------------+
|view|avg_price_per_view|
+----+------------------+
|  51|          15453.49|
|  75|          14853.91|
|  54|          14790.46|
|  53|          14249.34|
|  52|          14111.16|
|  55|          14020.97|
|  77|           13976.7|
|  76|          13931.62|
|  78|          13854.48|
|  59|          13414.46|
|  84|          13300.39|
|  58|          13099.39|
|  81|          13005.84|
|  82|          12969.49|
|  57|          12883.17|
|  56|          12824.58|
|  79|           12779.3|
|  60|          12582.33|
|  91|           12498.6|
|  83|          12457.42|
+----+------------------+
only showing top 20 rows

--- 0.25266098976135254 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [12]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
          
spark.sql("""SELECT
view, 
ROUND(AVG(price / view), 2) AS avg_price_per_view
FROM sales
GROUP BY view
HAVING AVG(price) > 350000
ORDER BY avg_price_per_view DESC""" ).show()


print(f" --- {(time.time() - start_time)} seconds ---")


+----+------------------+
|view|avg_price_per_view|
+----+------------------+
|  51|          15453.49|
|  75|          14853.91|
|  54|          14790.46|
|  53|          14249.34|
|  52|          14111.16|
|  55|          14020.97|
|  77|           13976.7|
|  76|          13931.62|
|  78|          13854.48|
|  59|          13414.46|
|  84|          13300.39|
|  58|          13099.39|
|  81|          13005.84|
|  82|          12969.49|
|  57|          12883.17|
|  56|          12824.58|
|  79|           12779.3|
|  60|          12582.33|
|  91|           12498.6|
|  83|          12457.42|
+----+------------------+
only showing top 20 rows

 --- 0.15116500854492188 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("p_home_sales")


In [14]:
# 11. Read the formatted parquet data.
p_df = spark.read.parquet("p_home_sales")

In [15]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView("p_sales")

In [16]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""SELECT
view, 
ROUND(AVG(price / view), 2) AS avg_price_per_view
FROM p_sales
GROUP BY view
HAVING AVG(price) > 350000
ORDER BY avg_price_per_view DESC""" ).show()


print(f" --- {(time.time() - start_time)} seconds ---")

+----+------------------+
|view|avg_price_per_view|
+----+------------------+
|  51|          15453.49|
|  75|          14853.91|
|  54|          14790.46|
|  53|          14249.34|
|  52|          14111.16|
|  55|          14020.97|
|  77|           13976.7|
|  76|          13931.62|
|  78|          13854.48|
|  59|          13414.46|
|  84|          13300.39|
|  58|          13099.39|
|  81|          13005.84|
|  82|          12969.49|
|  57|          12883.17|
|  56|          12824.58|
|  79|           12779.3|
|  60|          12582.33|
|  91|           12498.6|
|  83|          12457.42|
+----+------------------+
only showing top 20 rows

 --- 0.29305505752563477 seconds ---


In [17]:
# 14. Uncache sales temporary table.
spark.sql("uncache table sales")

DataFrame[]

In [18]:
#15. Confirm sales is no longer cached
spark.catalog.isCached("sales")


False

24/06/04 21:51:14 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
